In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Busselton,AU,57.00,broken clouds,-33.65,115.33,Observatory Guest House
1,1,Albany,US,50.00,light rain,42.60,-73.97,NaN
2,3,Nogliki,RU,46.60,overcast clouds,51.83,143.17,"Hotel ""Nogliki"""
3,5,Rikitea,PF,75.29,broken clouds,-23.12,-134.97,Pension Maro'i
4,6,Dossor,KZ,55.27,overcast clouds,47.53,52.98,"Gostinitsa Orpk ""Nauryz"""


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = bool(input("Do you want it to be raining? (yes/no) "))
snowing = bool (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 65
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Busselton,AU,57.00,broken clouds,-33.65,115.33,Observatory Guest House
1,1,Albany,US,50.00,light rain,42.60,-73.97,NaN
4,6,Dossor,KZ,55.27,overcast clouds,47.53,52.98,"Gostinitsa Orpk ""Nauryz"""
5,8,Ribeira Grande,PT,58.41,clear sky,38.52,-28.70,Quinta da Meia Eira
7,13,Hovd,MN,55.81,clear sky,48.01,91.64,Hotel Khovd
8,14,Yulara,AU,57.20,overcast clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
9,16,Port Elizabeth,ZA,55.40,clear sky,-33.92,25.57,39 On Nile Guest House
11,20,Coquimbo,CL,55.40,clear sky,-29.95,-71.34,Hotel San Juan
13,22,Mataura,NZ,55.76,overcast clouds,-46.19,168.86,Ellie's Villa
14,23,Lagoa,PT,57.61,clear sky,39.05,-27.98,Casa Das Faias


In [7]:
preferred_cities_df.count()

City_ID                147
City                   147
Country                146
Max Temp               147
Current Description    147
Lat                    147
Lng                    147
Hotel Name             132
dtype: int64

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,57.00,broken clouds,-33.65,115.33,
1,Albany,US,50.00,light rain,42.60,-73.97,
4,Dossor,KZ,55.27,overcast clouds,47.53,52.98,
5,Ribeira Grande,PT,58.41,clear sky,38.52,-28.70,
7,Hovd,MN,55.81,clear sky,48.01,91.64,
8,Yulara,AU,57.20,overcast clouds,-25.24,130.99,
9,Port Elizabeth,ZA,55.40,clear sky,-33.92,25.57,
11,Coquimbo,CL,55.40,clear sky,-29.95,-71.34,
13,Mataura,NZ,55.76,overcast clouds,-46.19,168.86,
14,Lagoa,PT,57.61,clear sky,39.05,-27.98,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,57.00,broken clouds,-33.65,115.33,Observatory Guest House
1,Albany,US,50.00,light rain,42.60,-73.97,
4,Dossor,KZ,55.27,overcast clouds,47.53,52.98,"Gostinitsa Orpk ""Nauryz"""
5,Ribeira Grande,PT,58.41,clear sky,38.52,-28.70,Quinta da Meia Eira
7,Hovd,MN,55.81,clear sky,48.01,91.64,Hotel Khovd
8,Yulara,AU,57.20,overcast clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
9,Port Elizabeth,ZA,55.40,clear sky,-33.92,25.57,39 On Nile Guest House
11,Coquimbo,CL,55.40,clear sky,-29.95,-71.34,Hotel San Juan
13,Mataura,NZ,55.76,overcast clouds,-46.19,168.86,Ellie's Villa
14,Lagoa,PT,57.61,clear sky,39.05,-27.98,Casa Das Faias


In [20]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))